In [1]:
import mne
import fooof
import numpy as np
import os.path as op
import pickle
import saflow
import saflow
import os.path as op
import pickle as pkl
import numpy as np
import pandas as pd
import os
import mne_bids
import mne
from sklearn.model_selection import permutation_test_score, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.pyplot as plt
import fooof
from statsmodels.stats.multitest import fdrcorrection
import random
from matplotlib.backends.backend_pdf import PdfPages
from saflow.utils import create_fnames
import os

/tmp/ipykernel_75152/3321975988.py:2: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  import fooof


In [15]:
def select_data(magic_dict, feature_type='psd_corrected', trial_type=['correct_commission'], ignore_bads=True):
    
    return X, y, groups, VTC, task


def select_trial(event_dict, trial_type=['correct_commission'], bad_how='any', inout_how='all', inout='INOUT_2575', verbose=False):
    if bad_how is None or bad_how == 'ignore':
        bad_epoch = False
    elif bad_how == 'last':
        bad_epoch = event_dict['bad_epoch']
    elif bad_how == 'any':
        bad_epoch = np.sum(event_dict['included_bad_epochs']) > 0

    if str(event_dict[inout]) != 'nan':
        if inout_how == 'all':
            n_uniques = len(np.unique(event_dict['included_INOUT'])) == 1
            unique_elements = np.unique(event_dict['included_INOUT'])
            uniques_inout = any(element in ['IN', 'OUT'] for element in unique_elements)
            retain_inout =  n_uniques & uniques_inout
        elif inout_how == 'last':
            retain_inout = event_dict[inout] in ['IN', 'OUT']
    else:
        retain_inout = False
    retain_type = event_dict['task'] in trial_type
    retain_epoch = retain_inout & retain_type & ~bad_epoch
    if verbose:
        print(f'Bad : {bad_epoch}, InOut : {retain_inout}, Type : {retain_type}, Retain : {retain_epoch}')
    return retain_epoch


def average_bands(psd, freq_bins, bands=saflow.FREQS):
    band_power = []
    for band in bands:
        band_mask = np.logical_and(freq_bins >= band[0], freq_bins <= band[1])
        band_power.append(np.mean(psd[band_mask]))
    return np.array(band_power)

[]
Adding trial 0 of run 03 of subject sub-04


ZeroDivisionError: division by zero

In [9]:
feature = 'fooof'
feature_fpath = op.join(saflow.BIDS_PATH, 'derivatives', feature)

feat_to_get = 'psd_corrected'
trial_type_to_get = ['correct_commission']

X = []
y = []
groups = []
VTC = []
task = []
for sub in sorted(os.listdir(feature_fpath)):
    for file in sorted(os.listdir(op.join(feature_fpath, sub, 'meg'))):
        print(file)
        if 'magic' in file:
            fname = op.join(feature_fpath, sub, 'meg', file)
            fname_output = fname.replace('.pkl', '_magic.pkl')
            with open(fname, 'rb') as f:
                data = pickle.load(f)
            # Reshape data
            n_chans = 270
            n_trials = int(len(data)/n_chans)
            data_reshaped = np.array(data).reshape(n_trials, n_chans)

            for trial_idx in range(data_reshaped.shape[0]):
                epoch_selected = select_trial(data_reshaped[trial_idx][0]['info'], trial_type=trial_type_to_get)
                if epoch_selected:
                    for chan_idx in range(data_reshaped.shape[1]):
                        psd = data_reshaped[trial_idx][chan_idx][feat_to_get]
                        #freq_bins = data_reshaped[trial_idx][chan_idx]['freq_bins']
                        print(psd.shape)
                        0/0
            

sub-04_task-gradCPT_meg.pkl
sub-04_task-gradCPT_run-02_meg.pkl
sub-04_task-gradCPT_run-02_meg_magic.pkl
(101,)


ZeroDivisionError: division by zero

In [14]:

band_power = average_bands(psd, freq_bins)
band_power.shape

(8,)

In [38]:
event_dict = data_reshaped[0][0]['info']
inout='INOUT_2575'
print(event_dict)
select_trial(event_dict, inout = 'INOUT_2575')

{'event_idx': 19, 't0_sample': 7685, 'VTC': 1.4243404892351406, 'task': 'correct_commission', 'RT': 0.8624374108221673, 'INOUT': 'OUT', 'INOUT_2575': 'OUT', 'INOUT_1090': nan, 'bad_epoch': True, 'included_bad_epochs': array([False, False, False, False, False,  True,  True,  True]), 'included_events_idx': [1, 3, 6, 8, 11, 14, 17, 19], 'included_VTC': array([2.69773833, 1.07826667, 0.74077676, 1.36547468, 0.75137772,
       0.4781026 , 0.40111338, 1.42434049]), 'included_task': array(['correct_commission', 'correct_commission', 'correct_commission',
       'correct_commission', 'correct_commission', 'correct_commission',
       'correct_commission', 'correct_commission'], dtype=object), 'included_RT': array([1.06108916, 0.80844944, 0.75580056, 0.85325427, 0.75745433,
       0.71482305, 0.70281263, 0.86243741]), 'included_INOUT': array(['OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT'],
      dtype=object), 'included_INOUT_2575': array([nan, nan, nan, 'OUT', 'OUT', 'OUT', 'OUT', 'OU

False

array([nan, nan, nan, 'OUT', 'OUT', 'OUT', 'OUT', 'OUT'], dtype=object)

NameError: name 'inout_how' is not defined

In [50]:
len(np.unique(event_dict[inout])) == 1 & np.unique(event_dict[inout]) in ['IN', 'OUT']

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [48]:
np.unique(event_dict[inout]) in ['IN', 'OUT']

True

# Deprec

In [ ]:
# Create Magic dict

feature = 'fooof'
feature_fpath = op.join(saflow.BIDS_PATH, 'derivatives', feature)


for sub in sorted(os.listdir(feature_fpath)):
    for run in saflow.BLOCS_LIST:
        run = '0' + str(run)
        fname = op.join(feature_fpath, sub, 'meg', f'{sub}_task-gradCPT_run-{run}_meg.pkl')
        fname_output = fname.replace('.pkl', '_magic.pkl')
        if not op.exists(fname_output):
            magic_dict = []
            with open(fname, 'rb') as f:
                data = pickle.load(f)
            print(magic_dict)
            for trial_idx in range(len(data['trial_fooofs'])):

                INOUT = data['info'][trial_idx]['INOUT']
                print(f'Adding trial {trial_idx} of run {run} of subject {sub}')
                for chan_idx in range(len(data['trial_fooofs'][trial_idx])):
                    psd_raw = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).power_spectrum
                    freq_bins = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).freqs
                    0/0
                    if INOUT == 'IN':
                        psd_corrected = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).power_spectrum - data['IN_fooofs'].get_fooof(chan_idx)._ap_fit
                    elif INOUT == 'OUT':
                        psd_corrected = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).power_spectrum - data['OUT_fooofs'].get_fooof(chan_idx)._ap_fit

                    psd_model_fit = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).fooofed_spectrum_
                    exponent = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).aperiodic_params_[-1]
                    offset = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).aperiodic_params_[0]
                    knee = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).aperiodic_params_[1]
                    r_squared = data['trial_fooofs'][trial_idx].get_fooof(chan_idx).r_squared_
                    
                    data_dict = {'psd_raw': psd_raw, 
                                'psd_corrected': psd_corrected, 
                                'psd_model_fit': psd_model_fit, 
                                'exponent': exponent, 
                                'offset': offset, 
                                'knee': knee, 
                                'r_squared': r_squared,
                                'info': data['info'][trial_idx],
                                'freq_bins': freq_bins}
                    
                    magic_dict.append(data_dict)
            fname_output = fname.replace('.pkl', '_magic.pkl')
            with open(fname_output, 'wb') as f:
                pickle.dump(magic_dict, f)


In [24]:
output_fname = op.join('/home/hyruuk/GitHub/cocolab/cc_saflow/tmp/', 'subject_averaged_fooofs.pkl')
with open(output_fname, 'rb') as f:
    fooof_array = pickle.load(f)

In [25]:
def average_bands_from_bands(fm, freqs=saflow.FREQS):
    fooof_freqs = fm.freqs
    original_bands = []
    fooof_bands = []
    original_corrected = []
    for band in freqs:
        freq_mask = np.where((fooof_freqs >= band[0]) & (fooof_freqs <= band[1]))[0]
        original = np.mean(fm.power_spectrum[freq_mask])
        original_bands.append(original)

        fooof = np.mean(fm.fooofed_spectrum_[freq_mask])
        fooof_bands.append(fooof)

        corrected = fm.power_spectrum - fm._ap_fit
        original_corrected.append(np.mean(corrected[freq_mask]))
    
    exponent = fm.get_params('aperiodic_params', 'exponent')
    offset = fm.get_params('aperiodic_params', 'offset')
    r_squared = fm.get_params('r_squared')
    return original_bands, fooof_bands, original_corrected, exponent, offset, r_squared

def get_data_matrix(fooof_array):
    original_bands_array = []
    fooof_bands_array = []
    original_corrected_array = []
    exponent_array = []
    offset_array = []
    r_squared_array = []

    for cond_idx in range(fooof_array.shape[0]):
        original_bands_subj = []
        fooof_bands_subj = []
        original_corrected_subj = []
        exponent_subj = []
        offset_subj = []
        r_squared_subj = []
        for subj_idx in range(fooof_array.shape[1]):
            original_bands_chans = []
            fooof_bands_chans = []
            original_corrected_chans = []
            exponent_chans = []
            offset_chans = []
            r_squared_chans = []
            for chan_idx in range(fooof_array.shape[2]):
                fm = fooof_array[cond_idx, subj_idx, chan_idx]
                original_bands, fooof_bands, original_corrected, exponent, offset, r_squared = average_bands_from_bands(fm)
                original_bands_chans.append(original_bands)
                fooof_bands_chans.append(fooof_bands)
                original_corrected_chans.append(original_corrected)
                exponent_chans.append(exponent)
                offset_chans.append(offset)
                r_squared_chans.append(r_squared)
            original_bands_subj.append(original_bands_chans)
            fooof_bands_subj.append(fooof_bands_chans)
            original_corrected_subj.append(original_corrected_chans)
            exponent_subj.append(exponent_chans)
            offset_subj.append(offset_chans)
            r_squared_subj.append(r_squared_chans)
        original_bands_array.append(original_bands_subj)
        fooof_bands_array.append(fooof_bands_subj)
        original_corrected_array.append(original_corrected_subj)
        exponent_array.append(exponent_subj)
        offset_array.append(offset_subj)
        r_squared_array.append(r_squared_subj)

        data = {'original_bands': np.array(original_bands_array),
                'fooof_bands': np.array(fooof_bands_array),
                'original_corrected': np.array(original_corrected_array),
                'exponent': np.array(exponent_array),
                'offset': np.array(offset_array),
                'r_squared': np.array(r_squared_array)}
    return data

data = get_data_matrix(fooof_array)

In [26]:
data['original_bands'].shape

(2, 32, 270, 8)

In [27]:
def format_data_for_classif(data, feature='original_bands'):
    X = []
    y = []
    groups = []
    for cond_idx in range(data[feature].shape[0]):
        for subj_idx in range(data[feature].shape[1]):
            X.append(data[feature][cond_idx, subj_idx])
            y.append(cond_idx)
            groups.append(subj_idx)
    X = np.array(X).transpose(2, 0, 1)
    y = np.array(y)
    groups = np.array(groups)
    return X, y, groups

X, y, groups = format_data_for_classif(data, feature='original_bands')

In [30]:
X_contrast, tvals, pvals = simple_contrast(X, y, groups)


NameError: name 'y_avg' is not defined

In [29]:
def simple_contrast(X, y, groups):
    """Computes subject-averages and contrasts between conditions."""
    n_features = X.shape[0]
    # Average each condition separately
    X_avg_by_cond = []
    for cond in np.unique(y):
        X_avg_by_cond.append(np.mean(X[y == cond], axis=0))
    # Compute normalized contrast (A - B)/B
    X_contrast = (X_avg_by_cond[0] - X_avg_by_cond[1]) / X_avg_by_cond[1]
    # Split conditions for ttest
    X_condA = X[y == 0]
    X_condB = X[y == 1]
    
    # Compute t-test
    tvals = []
    pvals = []
    for feature_idx in range(n_features):
        t, p = stats.ttest_rel(X_condA[:,feature_idx,:], X_condB[:,feature_idx,:], axis=0)
        tvals.append(t)
        pvals.append(p)
    tvals = np.array(tvals)
    pvals = np.array(pvals)
    return X_contrast, tvals, pvals


def singlefeat_classif(clf, cv, X, y, groups, n_perms=1):
    clf = LinearDiscriminantAnalysis()
    cv = LeaveOneGroupOut()

    all_scores, all_perm_scores, all_pvals = [], [], []
    for freq_idx in range(X.shape[0]):
        scores, perm_scores, pvals = [], [], []
        for chan_idx in range(X.shape[-1]):
            X_sf = X[freq_idx,:,chan_idx]
            score, permutation_scores, pvalue = permutation_test_score(clf, X=X_sf.reshape(-1, 1), y=y, groups=groups, cv=cv, n_permutations=n_perms, n_jobs=-1)
            scores.append(score)
            perm_scores.append(permutation_scores)
            pvals.append(pvalue)
            print(f'Computed feature {freq_idx} chan {chan_idx} Score {score}, pvalue {pvalue}')
        all_scores.append(scores)
        all_perm_scores.append(perm_scores)
        all_pvals.append(pvals)
        
    all_scores = np.array(all_scores)
    all_perm_scores = np.array(all_perm_scores)
    all_pvals = np.array(all_pvals)
    all_results = {'scores': all_scores, 'perm_scores': all_perm_scores, 'pvals': all_pvals}
    return all_results